In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

from src.models import get_device, load_model, get_preprocess, get_target_layer
from src.data import load_voc2007_val
from src.explainers import (
    GradCAM,
    saliency_map,
    integrated_gradients,
    lime_explanation
)
from src.visualization import render_attribution, bgr_to_rgb


In [ ]:
device = get_device()
model = load_model(device)
preprocess = get_preprocess()
target_layer = get_target_layer(model, layer_type="late")


In [ ]:
dataset = load_voc2007_val(root="./data", download=True)
print(f"Loaded VOC 2007 val set with {len(dataset)} images")

In [ ]:
# Choose a fixed image index (same as used in experiments)
idx = 0  # change only if you used a different fixed index

img_pil, gt_boxes = dataset[idx]

In [ ]:
# PIL → RGB numpy
img_rgb = np.array(img_pil)

# RGB → BGR for OpenCV-based visualization
img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

# Preprocess for model
input_tensor = preprocess(img_pil).unsqueeze(0).to(device)


In [ ]:
with torch.no_grad():
    outputs = model(input_tensor)
    target_class = outputs.argmax(dim=1).item()

print("Target class index:", target_class)


In [ ]:
sal_map = saliency_map(model, input_tensor, target_class)

In [ ]:
ig_map = integrated_gradients(
    model,
    input_tensor,
    target_class,
    baseline=None,
    steps=50
)

In [ ]:
lime_map = lime_explanation(
    model,
    preprocess,
    img_rgb,
    target_class,
    num_samples=1000
)

In [ ]:
methods = {
    "Grad-CAM": cam_map,
    "Saliency": sal_map,
    "Integrated Gradients": ig_map,
    "LIME": lime_map
}

plt.figure(figsize=(12, 8))

for i, (name, attr) in enumerate(methods.items()):
    overlay = render_attribution(img_bgr, attr, alpha=0.5)
    overlay_rgb = bgr_to_rgb(overlay)

    plt.subplot(2, 2, i + 1)
    plt.imshow(overlay_rgb)
    plt.title(name)
    plt.axis("off")

plt.tight_layout()
plt.show()
